In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 334045.28306914,  333592.30688303],
        [ 347818.07505192,  343680.09055295],
        [ 618147.37492337,  627342.55586818],
        [ 617422.56354646,  639685.37020618],
        [ 164340.77703813,  167598.29727564],
        [ 206595.51760183,  216214.45811829],
        [ 409195.82621593,  406549.74054826],
        [ 409148.09599309,  413267.68765339],
        [ 393814.30400124,  388356.35990397],
        [ 406155.57102336,  408211.3513791 ],
        [ 440862.15923249,  454172.36143204],
        [ 335373.09430677,  318038.92786086],
        [  95764.03325308,   98386.24936647],
        [  96009.6241983 ,   92386.5420172 ],
        [ 163155.01504804,  160238.68843344],
        [ 142133.75778526,  141851.56304812],
        [ 210692.49106262,  214161.92369329],
        [ 210887.52927761,  214756.13751233],
        [ 385674.74413159,  385276.97050591],
        [ 335615.97519585,  340468.47637927],
        [ 465285.10447507,  459577.73047257],
        [ 565146.75813738,  570983

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e-01, 8e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -7.67846592e+07  0.00000000e+00  5.86e+03 3.53e+03  1.00e+06     0s
   1  -6.42919694e+07 -1.67707413e+07  2.15e+03 1.30e+03  4.08e+05     0s
   2  -4.75051086e+07 -4.39591172e+07  2.20e+02 1.32e+02  9.71e+04     0s
   3  -4.54753356e+07 -4.59354526e+07  2.70e+00 1.63e+00  1.0

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.7780602584612308e-09,
 150723.95199906098,
 140970.03573476532,
 112243.811993874,
 103946.91906064985,
 6.469704730401238e-09,
 6.112134086677945e-09,
 158955.51862196013,
 1.720293170702691e-10,
 278650.263928456,
 1.7202931706496854e-10,
 1.7202931706837652e-10,
 1.675735400574652e-10,
 1.6843162535827106e-10,
 591000.3703217058,
 65901.208215353,
 8.328072389630506,
 190850.87141037692,
 533277.336659049,
 6.485039251354384e-09,
 6.092980450853172e-09,
 5.475273368035251e-06,
 459130.83500448114,
 1.0393399128568148e-07,
 1.0395606245559556e-07,
 1.0395605772156908e-07,
 88869.89368642715,
 206345.93842130434,
 0.29100766438097025,
 0.4215011219121152,
 458885.3840062396,
 17800.546139623297,
 118348.55320758594,
 179039.2259956533,
 139751.09678184445,
 6.670820850043411e-09,
 6.670820780748281e-09,
 6.664187841429906e-09,
 22112.363742030342,
 148748.63007289832,
 2.7316405275112534e-09,
 2.661699458396729e-09,
 1.4195152587027288e-08,
 1.4195152271434247e-08,
 1.4128982600318